In [ ]:
import os
import sys

# Obtenir le répertoire de travail courant
current_dir = os.getcwd()

# Obtenir le répertoire parent de `maps` (qui est `src`)
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))

# Ajouter `src` au chemin de recherche des modules
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [ ]:
import pandas as pd
import numpy as np
import folium.features

from useful_methods.data_processing import extract_data
from useful_methods.ihm.plots import plot_graph
from useful_methods.neighbours_delaunay.graphs import delaunay_graph
from useful_methods.neighbours_delaunay.miscellaneous_for_neighbouring import mean_distance_to_NN
from useful_methods.neighbours_delaunay.enhanced_criteria import *
from useful_methods.ihm.maps.mapUtils import *

In [ ]:
df = pd.read_csv("../../database/data.csv", sep=";", decimal=',')

In [ ]:
df_extracted = extract_data(df, provider='Orange', region='Normandie', techno='4g')

In [ ]:
mean_distance_params = {
    ']0, 1] km': {'colour': '#030464', 'angle': 40, 'distance': 2},
    ']1, 2] km': {'colour': '#069AF3', 'angle': 30, 'distance': 5},
    ']2, 4] km': {'colour': '#02D4BB', 'angle': 25, 'distance': 10},
    ']4, inf] km': {'colour': '#0DBF75', 'angle': 15, 'distance': 15},
}

## With Delaunay

In [ ]:
G, pos = delaunay_graph(df_extracted)

In [ ]:
mean_distances = mean_distance_to_NN(df_extracted[['x', 'y']], n_neighbours=3) # 3 to have more neighbours

In [ ]:
G_tot = distance_criterion_enhanced(G, pos, params=mean_distance_params, mean_distance_to_NN=mean_distances)
G_tot = angle_criterion_enhanced(G_tot, pos, params=mean_distance_params, mean_distance_to_NN=mean_distances)
G_tot = quadrant_criterion_enhanced(G_tot, pos)

In [ ]:
plot_graph(G_tot, pos)

In [ ]:
len(G_tot.edges)

## Without Delaunay

In [ ]:
points_pos = df_extracted[['latitude', 'longitude']].values

G_other = nx.Graph()
nodes = df_extracted.index
G_other.add_nodes_from(nodes)

pos_other = dict(zip(df_extracted.index, points_pos))

In [ ]:
coordsXY = df_extracted[['x', 'y']]
n_neighbours = 3

nbrs = NearestNeighbors(n_neighbors=n_neighbours+1, metric='euclidean').fit(coordsXY)
distances, _ = nbrs.kneighbors(coordsXY)

data_mean_dist = np.mean(distances[:, 1:]/1000, axis=1)  # we exclude the first element (distance to ourself is 0)
mean_distances_other = pd.Series(data=data_mean_dist, index=coordsXY.index)

In [ ]:
for node in tqdm(pos.keys(), desc="nodes - distance"):
    max_distance = mean_distance_choice(node, mean_distances, mean_distance_params, 'distance')

    print(max_distance)
    for neigh_ind in coordsXY.iloc[nbrs.radius_neighbors([coordsXY.loc[node]], radius=max_distance*1000, sort_results=True)[1][0][1:]].index:
        edge = [node, neigh_ind]
        G_other.add_edges_from([edge])

In [ ]:
G_other = angle_criterion_enhanced(G_other, pos, params=mean_distance_params, mean_distance_to_NN=mean_distances)
G_other = quadrant_criterion_enhanced(G_other, pos)

In [ ]:
plot_graph(G_other, pos)

In [ ]:
len(G_other.edges)

## On a map

In [ ]:
def add_graph_edges(G_base: nx.Graph, G: nx.Graph, pos: dict, fg: folium.FeatureGroup, colour: str):
    for edge in G_base.edges:
        stations = []
        if(not(edge in G.edges)):
            stations.append(pos[edge[0]])
            stations.append(pos[edge[1]])

            folium.PolyLine(np.array(stations), color=colour, weight=2.5, opacity=1).add_to(fg)

In [ ]:
map = folium.Map(location=np.mean(df_extracted[['latitude','longitude']], axis=0), zoom_start=8.5, tiles="Cartodb Positron")
edges_tot = folium.FeatureGroup(f"Edges - total ({len(G_tot.edges)})").add_to(map)
edges_other = folium.FeatureGroup(f"Edges - other ({len(G_other.edges)})").add_to(map)
points = folium.FeatureGroup(f"Points ({len(G)})").add_to(map)

add_graph_edges(G_other, nx.Graph(), pos, edges_other, colour="orange")
add_graph_edges(G_tot, nx.Graph(), pos, edges_tot, colour="#AAA662")

for ind, latitude, longitude in df_extracted[['latitude', 'longitude']].itertuples():
    color = mean_distance_choice(ind, mean_distances, mean_distance_params, 'colour')
    points.add_child(folium.CircleMarker(location=[latitude, longitude], color=color, radius=2.5, popup=f"mean_dist={mean_distances.get(ind)}"))
folium.LayerControl().add_to(map)

addLegend(map, mean_distance_params)

map.save("../../out/maps/neighbourhood_nextGen.html")